In [2]:
import numpy as np
import pandas as pd

# IDK, but importing BERTopic without somthing from torch ends with error
# I think it's reated to broken depndencies with dask in datamapplot library
from torch import device
from bertopic import BERTopic
import datamapplot

import gc
import os
os.chdir("/home/denisalpino/dev/FinABYSS/")

# Data loading

Load pre-saved 2d embeddings and them clustrer labels

In [3]:
embeddings_2d = np.load("embeddings2d.npy")
labels = np.load("labels.npy")

Calculate number of embeddings to get indexes for extraction correspondent rows from corpus

In [4]:
# Load full-sized embeddings to get indexes
embeddings_full = np.load('data/preprocessed/embeddings_mp.npy')
rows, _ = embeddings_full.shape

# Release memory
del embeddings_full
gc.collect();

# Set random state
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Get indexes
embeddings_idx = np.random.choice(rows, size=int(rows * 0.10), replace=False)

# Get needed articles with their metadata
df = pd.read_parquet("data/preprocessed/articles.parquet").iloc[embeddings_idx].reset_index(drop=True)

Load topic model for cluster names extraction

In [5]:
topic_model = BERTopic.load("topic_model")

2025-06-01 22:48:45,620 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


### Preprocessing

Get custom generated topics from BERTopic for each article

In [6]:
# Number of cluster for each article
topic_per_doc = topic_model.topics_

# Get unique labels
unique_topics = set(topic_per_doc)

# Get generated name for each cluster
names = [topic_model.custom_labels_[topic + topic_model._outliers] for topic in unique_topics]
# BUG: there is a problem with it cause few topics have the same names
topic_name_mapping = {topic_num: topic_name for topic_num, topic_name in zip(unique_topics, names)}

# Change noise label for a datamapplot
topic_name_mapping[-1] = "Unlabelled"

# Get a array of correspondent topic name for each article
named_topic_per_doc = pd.Series(topic_per_doc).map(topic_name_mapping).values

Create DataFrame wth formatted and neded data special for vizualization

In [7]:
data = []
for x, y, dt, topic, text, title, source, tickers, url in zip(
        embeddings_2d[:, 0],
        embeddings_2d[:, 1],
        df["datetime"],
        named_topic_per_doc,
        df["text"],
        df["title"],
        df["source"],
        df["assets"],
        df["url"]
    ):
    # Don't use text of articles labeled as noise for saving RAM during usage web-map
    text = text if topic != "Unlabelled" else ""
    tickers = tickers if tickers.size > 0 else "—"
    datetime_formatted = dt.strftime("Published on %B %d, %Y at %H:%M:%S UTC")
    row = [x, y, title, topic, dt, datetime_formatted, len(text), url, ', '.join(tickers), source, text]
    data.append(row)

df2 = pd.DataFrame(data, columns=["x", "y", "title", "topic", "datetime", "datetime_formatted", "char_len", "link", "tickers", "source", "text"])
df2["datetime"] = pd.to_datetime(df2["datetime"])

# Release memory
del df, data, named_topic_per_doc, topic_name_mapping, names, topic_per_doc
gc.collect();

# Configuring web-page (HTML), styles (CSS) and actions (JavaScript) for Semantic Map

Custom styles for pop-ups when user hover over dot

In [8]:
badge_css = """
    border-radius:6px;
    width:fit-content;
    max-width:75%;
    margin:2px;
    padding: 2px 10px 2px 10px;
    font-size: 10pt;
"""
hover_text_template = f"""
<div>
    <div style="font-size:12pt;padding:2px;"><b>{{title}}</b></div>
    <div style="background-color:#0000FF;{badge_css}">{{topic}}</div>
    <div style="background-color:#FF0000;{badge_css}">{{tickers}}</div>
    <div>SOURCE: {{source}}</div>
    <div>{{datetime_formatted}}</div>
</div>
"""

Get unique sources, topics and tickers for Filtering Menu (aka legend) and sort them

In [9]:
sources = df2['source'].value_counts().index
topics = df2['topic'].value_counts().index.sort_values()

tickers = []
for tkrs in df2['tickers'].values:
    tickers.extend(tkrs.split(", "))
tickers = pd.Series(tickers).value_counts()[1:].index.sort_values()

Develop custom plugins (html, css, js)

In [10]:
# Hard-code logo to exclude any dependencies
svg_html = """
<svg viewBox="0 0 2306 324" fill="none" xmlns="http://www.w3.org/2000/svg">
  <g clip-path="url(#clip0_40_808)">
    <ellipse cx="267.5" cy="161" rx="6.5" ry="5" fill="white"/>
    <path d="M34.5781 214.148C34.5781 104.942 122.907 16.5737 232.065 16.5737H231.6C298.893 16.5737 353.345 71.0502 353.345 138.372C353.345 180.256 319.468 214.148 277.603 214.148H278.068C252.384 214.148 231.6 193.355 231.6 167.66C231.6 151.472 244.694 138.372 260.875 138.372H260.41C270.17 138.372 278.068 146.273 278.068 156.038C278.068 162.461 272.872 167.66 266.451 167.66C263.369 167.66 260.875 164.957 260.875 161.616" stroke="white" stroke-width="5" stroke-miterlimit="10" stroke-linecap="round" stroke-linejoin="round"/>
    <circle cx="34" cy="224" r="15" fill="white"/>
    <ellipse cx="236.5" cy="16.4999" rx="12.5" ry="12.5055" fill="white"/>
    <ellipse cx="353" cy="138" rx="10" ry="10.0044" fill="white"/>
    <ellipse cx="278.5" cy="214.5" rx="7.5" ry="7.50331" fill="white"/>
    <ellipse cx="231" cy="171" rx="5" ry="5.00221" fill="white"/>
    <path d="M320.623 394.025C211.466 394.025 123.137 305.657 123.137 196.451V196.916C123.137 129.594 177.589 75.1175 244.882 75.1175C286.747 75.1175 320.623 109.009 320.623 150.893V150.428C320.623 176.123 299.84 196.916 274.156 196.916C257.975 196.916 244.882 183.817 244.882 167.629V168.093C244.882 158.329 252.779 150.428 262.539 150.428C268.96 150.428 274.156 155.626 274.156 162.05C274.156 165.134 271.454 167.629 268.115 167.629" stroke="white" stroke-width="2" stroke-miterlimit="10" stroke-linecap="round" stroke-linejoin="round"/>
    <ellipse cx="157.295" cy="110.899" rx="5.811" ry="5.80843" transform="rotate(-90 157.295 110.899)" fill="white"/>
    <ellipse cx="297.096" cy="95.7609" rx="4.6488" ry="4.64675" transform="rotate(-90 297.096 95.7609)" fill="white"/>
    <ellipse cx="310.655" cy="179.711" rx="3.4866" ry="3.48506" transform="rotate(-90 310.655 179.711)" fill="white"/>
    <ellipse cx="254.855" cy="189.533" rx="2.3244" ry="2.32337" transform="rotate(-90 254.855 189.533)" fill="white"/>
    <path d="M500.422 107.853C500.422 217.058 412.093 305.427 302.935 305.427H303.4C236.107 305.427 181.655 250.95 181.655 183.628C181.655 141.744 215.532 107.853 257.397 107.853H256.932C282.616 107.853 303.4 128.645 303.4 154.341C303.4 170.529 290.306 183.628 274.125 183.628H274.59C264.83 183.628 256.932 175.727 256.932 165.963C256.932 159.539 262.128 154.341 268.549 154.341C271.631 154.341 274.125 157.043 274.125 160.384" stroke="white" stroke-width="5" stroke-miterlimit="10" stroke-linecap="round" stroke-linejoin="round"/>
    <ellipse cx="499" cy="99" rx="15" ry="15.0066" transform="rotate(-180 499 99)" fill="white"/>
    <ellipse cx="298.5" cy="305.5" rx="12.5" ry="12.5055" transform="rotate(-180 298.5 305.5)" fill="white"/>
    <ellipse cx="181" cy="184" rx="10" ry="10.0044" transform="rotate(-180 181 184)" fill="white"/>
    <ellipse cx="255.5" cy="107.5" rx="7.5" ry="7.50331" transform="rotate(-180 255.5 107.5)" fill="white"/>
    <ellipse cx="303" cy="151" rx="5" ry="5.00221" transform="rotate(-180 303 151)" fill="white"/>
    <path d="M214.377 -72.0249C323.534 -72.0249 411.863 16.3432 411.863 125.549V125.084C411.863 192.406 357.411 246.883 290.118 246.883C248.253 246.883 214.377 212.991 214.377 171.107V171.572C214.377 145.877 235.16 125.084 260.844 125.084C277.025 125.084 290.118 138.184 290.118 154.372V153.907C290.118 163.671 282.221 171.572 272.461 171.572C266.04 171.572 260.844 166.374 260.844 159.95C260.844 156.867 263.546 154.372 266.885 154.372" stroke="white" stroke-width="2" stroke-miterlimit="10" stroke-linecap="round" stroke-linejoin="round"/>
    <ellipse cx="376.809" cy="210.811" rx="5.811" ry="5.80843" transform="rotate(90 376.809 210.811)" fill="white"/>
    <ellipse cx="237.646" cy="225.649" rx="4.6488" ry="4.64675" transform="rotate(90 237.646 225.649)" fill="white"/>
    <ellipse cx="225.486" cy="141.487" rx="3.4866" ry="3.48506" transform="rotate(90 225.486 141.487)" fill="white"/>
    <ellipse cx="280.323" cy="132.324" rx="2.3244" ry="2.32337" transform="rotate(90 280.323 132.324)" fill="white"/>
    <circle cx="268" cy="161" r="5" transform="rotate(-90 268 161)" fill="black"/>
  </g>
  <path d="M801.38 104C803.513 104 805.18 104.667 806.38 106C807.713 107.2 808.38 108.867 808.38 111V113.8C808.38 115.933 807.713 117.667 806.38 119C805.18 120.2 803.513 120.8 801.38 120.8H647.58V168.4H789.58C794.246 168.4 796.58 170.733 796.58 175.4V178.4C796.58 183.067 794.246 185.4 789.58 185.4H647.58V237C647.58 241.667 645.246 244 640.58 244H636.18C631.513 244 629.18 241.667 629.18 237V111C629.18 106.333 631.513 104 636.18 104H801.38ZM862.528 104C865.861 104 867.528 105.667 867.528 109V115.2C867.528 118.533 865.861 120.2 862.528 120.2H855.328C851.995 120.2 850.328 118.533 850.328 115.2V109C850.328 105.667 851.995 104 855.328 104H862.528ZM862.528 142.4C865.861 142.4 867.528 144.067 867.528 147.4V239C867.528 242.333 865.861 244 862.528 244H855.328C851.995 244 850.328 242.333 850.328 239V147.4C850.328 144.067 851.995 142.4 855.328 142.4H862.528ZM987.339 141C1009.07 141 1024.61 145.133 1033.94 153.4C1043.27 161.667 1047.94 174.333 1047.94 191.4V239C1047.94 242.333 1046.27 244 1042.94 244H1035.74C1032.41 244 1030.74 242.333 1030.74 239V191.4C1030.74 178.6 1027.07 169.667 1019.74 164.6C1012.41 159.533 1000.41 157 983.739 157C966.006 157 952.406 159.867 942.939 165.6C933.472 171.333 928.739 179.933 928.739 191.4V239C928.739 242.333 927.072 244 923.739 244H916.539C913.206 244 911.539 242.333 911.539 239V147.4C911.539 144.067 913.206 142.4 916.539 142.4H923.739C927.072 142.4 928.739 144.067 928.739 147.4V158.4C940.739 146.8 960.272 141 987.339 141ZM1304.34 235.6C1306.07 237.867 1306.47 239.867 1305.54 241.6C1304.74 243.2 1302.87 244 1299.94 244H1293.74C1290.54 244 1287.74 242.533 1285.34 239.6L1265.74 213H1124.14L1104.54 239.6C1102.14 242.533 1099.34 244 1096.14 244H1089.94C1087 244 1085.07 243.2 1084.14 241.6C1083.34 239.867 1083.8 237.867 1085.54 235.6L1179.54 108C1181.67 105.067 1184.6 103.6 1188.34 103.6H1201.54C1205.27 103.6 1208.2 105.067 1210.34 108L1304.34 235.6ZM1136.14 196.4H1253.74L1194.94 116.4L1136.14 196.4ZM1511.72 168.6C1530.26 174.467 1539.52 186.2 1539.52 203.8C1539.52 230.6 1520.66 244 1482.92 244H1351.92C1347.26 244 1344.92 241.667 1344.92 237V111C1344.92 106.333 1347.26 104 1351.92 104H1475.52C1485.12 104 1493.39 105.133 1500.32 107.4C1507.39 109.667 1512.72 112.733 1516.32 116.6C1520.06 120.467 1522.72 124.533 1524.32 128.8C1526.06 133.067 1526.92 137.6 1526.92 142.4C1526.92 154 1521.86 162.733 1511.72 168.6ZM1475.92 120.6H1363.32V164.8H1475.92C1486.86 164.8 1495.12 162.8 1500.72 158.8C1506.32 154.8 1509.12 149.267 1509.12 142.2C1509.12 134.733 1506.39 129.267 1500.92 125.8C1495.59 122.333 1487.26 120.6 1475.92 120.6ZM1481.52 227.4C1495.79 227.4 1505.92 225.733 1511.92 222.4C1518.06 218.933 1521.12 212.933 1521.12 204.4C1521.12 196 1518.06 190.067 1511.92 186.6C1505.79 183 1495.66 181.2 1481.52 181.2H1363.32V227.4H1481.52ZM1752.84 104C1756.04 104 1757.97 104.867 1758.64 106.6C1759.44 108.333 1758.57 110.333 1756.04 112.6L1669.64 190.6V237C1669.64 241.667 1667.3 244 1662.64 244H1658.24C1653.57 244 1651.24 241.667 1651.24 237V191.4L1564.24 112.6C1561.84 110.333 1560.97 108.333 1561.64 106.6C1562.3 104.867 1564.24 104 1567.44 104H1576.04C1579.37 104 1582.3 105.067 1584.84 107.2L1659.04 174.4H1661.44L1735.44 107.2C1737.97 105.067 1740.9 104 1744.24 104H1752.84ZM1807.4 139C1807.4 142.6 1809.27 145.8 1813 148.6C1816.74 151.267 1822.34 153.6 1829.8 155.6C1837.4 157.467 1844.6 158.933 1851.4 160C1858.2 161.067 1866.8 162.2 1877.2 163.4C1884.94 164.333 1890.94 165.067 1895.2 165.6C1899.6 166.133 1905.34 166.933 1912.4 168C1919.6 168.933 1925.27 169.933 1929.4 171C1933.67 172.067 1938.6 173.333 1944.2 174.8C1949.8 176.267 1954.2 177.933 1957.4 179.8C1960.6 181.667 1963.74 183.8 1966.8 186.2C1970 188.6 1972.27 191.333 1973.6 194.4C1975.07 197.467 1975.8 200.8 1975.8 204.4C1975.8 219.067 1967.94 229.733 1952.2 236.4C1936.6 243.067 1912.27 246.4 1879.2 246.4H1876.8C1856.8 246.4 1838.94 244.667 1823.2 241.2C1807.6 237.733 1796.67 234.067 1790.4 230.2C1788.27 228.867 1787.07 227.333 1786.8 225.6C1786.67 223.733 1787.47 221.933 1789.2 220.2L1791.6 218C1794.27 215.333 1797.47 214.8 1801.2 216.4C1805.2 218.133 1810.2 219.933 1816.2 221.8C1822.34 223.533 1831.14 225.4 1842.6 227.4C1854.2 229.267 1865.6 230.2 1876.8 230.2H1879.2C1905.34 230.2 1924.74 228 1937.4 223.6C1950.07 219.2 1956.4 213.4 1956.4 206.2C1956.4 204.067 1955.87 202.133 1954.8 200.4C1953.74 198.533 1952 196.867 1949.6 195.4C1947.2 193.933 1944.74 192.667 1942.2 191.6C1939.8 190.4 1936.4 189.333 1932 188.4C1927.6 187.333 1923.74 186.467 1920.4 185.8C1917.2 185.133 1912.74 184.467 1907 183.8C1901.27 183 1896.6 182.4 1893 182C1889.54 181.6 1884.6 181.067 1878.2 180.4C1870.47 179.6 1863.54 178.733 1857.4 177.8C1851.4 176.867 1844.67 175.667 1837.2 174.2C1829.74 172.733 1823.27 171 1817.8 169C1812.47 166.867 1807.4 164.467 1802.6 161.8C1797.8 159 1794.14 155.733 1791.6 152C1789.2 148.133 1788 143.867 1788 139.2C1788 126.267 1795.07 116.8 1809.2 110.8C1823.34 104.667 1845.34 101.6 1875.2 101.6H1880.2C1899.8 101.6 1917 103.333 1931.8 106.8C1946.6 110.267 1957.07 113.933 1963.2 117.8C1967.87 120.6 1968.27 123.933 1964.4 127.8L1962.2 130C1959.54 132.667 1956.27 133.2 1952.4 131.6C1948.54 129.867 1943.74 128.067 1938 126.2C1932.4 124.333 1924.14 122.467 1913.2 120.6C1902.27 118.733 1891.27 117.8 1880.2 117.8H1875.2C1851.34 117.8 1834.07 119.6 1823.4 123.2C1812.74 126.667 1807.4 131.933 1807.4 139ZM2034.44 139C2034.44 142.6 2036.3 145.8 2040.04 148.6C2043.77 151.267 2049.37 153.6 2056.84 155.6C2064.44 157.467 2071.64 158.933 2078.44 160C2085.24 161.067 2093.84 162.2 2104.24 163.4C2111.97 164.333 2117.97 165.067 2122.24 165.6C2126.64 166.133 2132.37 166.933 2139.44 168C2146.64 168.933 2152.3 169.933 2156.44 171C2160.7 172.067 2165.64 173.333 2171.24 174.8C2176.84 176.267 2181.24 177.933 2184.44 179.8C2187.64 181.667 2190.77 183.8 2193.84 186.2C2197.04 188.6 2199.3 191.333 2200.64 194.4C2202.1 197.467 2202.84 200.8 2202.84 204.4C2202.84 219.067 2194.97 229.733 2179.24 236.4C2163.64 243.067 2139.3 246.4 2106.24 246.4H2103.84C2083.84 246.4 2065.97 244.667 2050.24 241.2C2034.64 237.733 2023.7 234.067 2017.44 230.2C2015.3 228.867 2014.1 227.333 2013.84 225.6C2013.7 223.733 2014.5 221.933 2016.24 220.2L2018.64 218C2021.3 215.333 2024.5 214.8 2028.24 216.4C2032.24 218.133 2037.24 219.933 2043.24 221.8C2049.37 223.533 2058.17 225.4 2069.64 227.4C2081.24 229.267 2092.64 230.2 2103.84 230.2H2106.24C2132.37 230.2 2151.77 228 2164.44 223.6C2177.1 219.2 2183.44 213.4 2183.44 206.2C2183.44 204.067 2182.9 202.133 2181.84 200.4C2180.77 198.533 2179.04 196.867 2176.64 195.4C2174.24 193.933 2171.77 192.667 2169.24 191.6C2166.84 190.4 2163.44 189.333 2159.04 188.4C2154.64 187.333 2150.77 186.467 2147.44 185.8C2144.24 185.133 2139.77 184.467 2134.04 183.8C2128.3 183 2123.64 182.4 2120.04 182C2116.57 181.6 2111.64 181.067 2105.24 180.4C2097.5 179.6 2090.57 178.733 2084.44 177.8C2078.44 176.867 2071.7 175.667 2064.24 174.2C2056.77 172.733 2050.3 171 2044.84 169C2039.5 166.867 2034.44 164.467 2029.64 161.8C2024.84 159 2021.17 155.733 2018.64 152C2016.24 148.133 2015.04 143.867 2015.04 139.2C2015.04 126.267 2022.1 116.8 2036.24 110.8C2050.37 104.667 2072.37 101.6 2102.24 101.6H2107.24C2126.84 101.6 2144.04 103.333 2158.84 106.8C2173.64 110.267 2184.1 113.933 2190.24 117.8C2194.9 120.6 2195.3 123.933 2191.44 127.8L2189.24 130C2186.57 132.667 2183.3 133.2 2179.44 131.6C2175.57 129.867 2170.77 128.067 2165.04 126.2C2159.44 124.333 2151.17 122.467 2140.24 120.6C2129.3 118.733 2118.3 117.8 2107.24 117.8H2102.24C2078.37 117.8 2061.1 119.6 2050.44 123.2C2039.77 126.667 2034.44 131.933 2034.44 139Z" fill="white"/>
  <defs>
    <clipPath id="clip0_40_808">
      <rect width="511" height="324" fill="white" transform="translate(12)"/>
    </clipPath>
  </defs>
</svg>
"""

# HTML with logo, and Filtering Menu (aka legend)
custom_html = f"""
<div id="logo-container" class="container-box">
  {svg_html}
</div>

<div id="legend">
  <div class="toggle-group" id="legend-toggle">
    <input type="radio" name="filter" id="filter-source"  value="source"  checked>
    <label for="filter-source">Source</label>
    <input type="radio" name="filter" id="filter-topic"   value="topic">
    <label for="filter-topic">Topic</label>
    <input type="radio" name="filter" id="filter-ticker"  value="tickers">
    <label for="filter-ticker">Ticker</label>
  </div>

  <!-- framed wrapper -->
  <div id="legend-inner">
    <div id="legend-source" class="legend-block">
      <!-- Search bar -->
      <input type="text" class="filter-search" placeholder="Search Source..." data-field="source">
      <!-- Container for “tags” of selected sources -->
      <div class="selected-tags" data-field="source"></div>
"""

for source in sources:
    custom_html += f"""
  <div class="row">
    <div id="{source}" class="box" style="background-color:#585858;"></div>
    <span class="item-label">{source}</span>
  </div>
"""
custom_html += """
    </div>

    <div id="legend-topic" class="legend-block" style="display:none;">
      <!-- Search bar -->
      <input type="text" class="filter-search" placeholder="Search Topic..." data-field="topic">
      <!-- Container for “tags” of selected topics -->
      <div class="selected-tags" data-field="topic"></div>
"""

for topic in topics:
    custom_html += f"""
  <div class="row">
    <div id="{topic}" class="box" style="background-color:#585858;"></div>
    <span class="item-label">{topic}</span>
  </div>
"""
custom_html += """
    </div>

    <div id="legend-ticker" class="legend-block" style="display:none;">
      <!-- Search bar -->
      <input type="text" class="filter-search" placeholder="Search Ticker..." data-field="tickers">
      <!-- Container for “tags” of selected tickers -->
      <div class="selected-tags" data-field="tickers"></div>
"""

for ticker in tickers:
    custom_html += f"""
  <div class="row">
    <div id="{ticker}" class="box" style="background-color:#585858;"></div>
    <span class="item-label">{ticker}</span>
  </div>
"""
custom_html += """
    </div>
  </div>
</div>
"""

# Custom CSS styles for Filter Menu (aka legend), logo, search bar etc
custom_css = """
*,
*::before,
*::after {
  font-family: "Montserrat" !important;
}

html, body, * {
  font-family: "Montserrat" !important;
}

/* === Root Settings and Font Inheritance === */
:root {
  font-size: 1rem; /* базовый размер ≃16px */
}

#legend,
#legend * {
  box-sizing: border-box;
}

/* === CONTAINER OF LEGEND === */
#legend {
  position: fixed;
  top: 2vh;
  bottom: 2vh;
  right: 2vw;
  width: 15vw;
  display: flex;
  flex-direction: column;
  border: 0.1rem solid #ccc;
  border-radius: 0.5rem;
  box-shadow: 0 0.125rem 0.375rem rgba(0, 0, 0, 0.15);
  padding: 0.75rem;
  background-color: rgba(0, 0, 0, 0.75);
}

/* Undo any shadow styles that may have been added previously */
#legend::before,
#legend::after {
  content: none;
}

/* === SWITCH STYLES (SOURCE / TOPIC / TICKER) === */
.toggle-group {
  display: flex;
  justify-content: space-between;
  gap: 0.5rem;
  margin-bottom: 0.75rem;
}

.toggle-group input {
  display: none;
}

.toggle-group label {
  flex: 1;
  text-align: center;
  padding: 0.375rem 0;
  border-radius: 0.75rem;
  cursor: pointer;
  background: #444;
  color: #ccc;
  font-size: 0.875rem;
  transition: background 0.2s, color 0.2s;
}

.toggle-group input:checked + label {
  background: #fff;
  color: #000;
  font-weight: 600;
}

/* === INTERNAL SCROLL LIST === */
#legend-inner {
  flex: 1 1 auto;
  position: relative;
  overflow-y: auto;
  padding-right: 0.5rem;
  margin-top: 0.25rem;
}

/* Undo any shadow styles of #legend-inner. */
#legend-inner::before,
#legend-inner::after {
  content: none;
}

/* Scrollbar for #legend-inner */
#legend-inner::-webkit-scrollbar {
  width: 0.5rem;
}
#legend-inner::-webkit-scrollbar-track {
  background: transparent;
}
#legend-inner::-webkit-scrollbar-thumb {
  background-color: rgba(50, 50, 50, 0.8);
  border-radius: 0.25rem;
}
/* Firefox */
#legend-inner {
  scrollbar-width: thin;
  scrollbar-color: rgba(50, 50, 50, 0.8) transparent;
}

/* === STYLES OF ROW and QUADRATES (BOX) === */
.row {
  display: flex;
  align-items: center;
  padding: 0.25rem 0;
  overflow-wrap: break-word;
  white-space: normal;
  font-size: 0.875rem;
  color: #eee;
}

.box {
  flex: 0 0 auto;
  height: 0.625rem;
  width: 0.625rem;
  border-radius: 0.125rem;
  margin-right: 0.3125rem;
  cursor: pointer;
  user-select: none;
  color: white;
  font-size: 0.875rem;
  text-align: center;
  line-height: 0.625rem;
}

/* The label to the right of the square */
.item-label {
  flex: 1;
  font-size: 0.875rem;
  color: #eee;
}

/* === SEARCH BAR STYLES === */
.filter-search {
  width: 100%;
  padding: 0.3125rem 0.5rem;
  margin-bottom: 0.5rem;
  margin-top: 0.25rem;
  border: 0.0625rem solid #666; /* ≃1px */
  border-radius: 0.25rem;      /* ≃4px */
  background: #222;
  color: #fff;
  font-size: 0.875rem;         /* ≃14px */
}

.filter-search::placeholder {
  color: #888;
}

/* === STYLES OF THE SELECTED “TAGS” CONTAINER === */
.selected-tags {
  display: flex;
  flex-wrap: wrap;
  gap: 0.375rem;
  margin-bottom: 0.5rem;
  max-height: 5rem;
  overflow-y: auto;
  padding-bottom: 0.25rem; /* ≃4px */
}

/* Scrollbar for .selected-tags */
.selected-tags::-webkit-scrollbar {
  width: 0.375rem;
}
.selected-tags::-webkit-scrollbar-track {
  background: transparent;
}
.selected-tags::-webkit-scrollbar-thumb {
  background-color: rgba(200, 200, 200, 0.6);
  border-radius: 0.1875rem;
}
/* Firefox */
.selected-tags {
  scrollbar-width: thin;
  scrollbar-color: rgba(200, 200, 200, 0.6) transparent;
}

/* === INDIVIDUAL “TAG” STYLES (SELECTED FILTER TAG) === */
.tag {
  display: inline-flex;
  align-items: center;
  gap: 0.25rem;
  padding: 0.125rem 0.375rem;
  background: #555;
  color: #fff;
  font-size: 0.75rem;
  border-radius: 0.75rem;
  white-space: nowrap;
}

.tag .tag-text {
  max-width: 6.25rem;
  overflow: hidden;
  text-overflow: ellipsis;
}

.tag .tag-remove {
  cursor: pointer;
  font-weight: bold;
}

/* === STYLES OF THE GENERAL SEARCH FIELD (TEXT-SEARCH) === */
#search-container {
  position: fixed;
  width: 27.7778vw;
  height: 3.125rem;  /* ≃50px */
  left: 2vw;
  top: 12vh
}

#text-search {
  width: 100%;
  height: 100%;
  padding: 0.5rem 2rem;  /* ≃8px / 16px */
  font-size: 1rem;      /* ≃16px */
  border: 0.0625rem solid #666; /* ≃1px */
  border-radius: 2rem;
  background: #222;
  color: #fff;
}

#text-search::placeholder {
  color: #888;
}

/* === LOGO CONTAINER STYLES === */
#logo-container {
  position: fixed;
  top: 2vh;
  left: 2vw;
  width: 27.7778vw;
  height: auto;
  z-index: 0;
}
"""

# Custom JavaScript code for filteering
custom_js = """
document.addEventListener("DOMContentLoaded", function() {
  const input = document.getElementById("text-search");
  if (input) {
    input.placeholder = "🔍 Search in financial news...";
  }
});

(function adaptiveSmoothMouseWheelZoom() {
  function tryAttach() {
    const container = document.getElementById("deck-container");
    if (!container) {
      return setTimeout(tryAttach, 100);
    }
    container.addEventListener("wheel", function(e) {
      if (e._isCustomZoomEvent) return;

      const absDeltaY = Math.abs(e.deltaY);
      const absDeltaX = Math.abs(e.deltaX);
      const MOUSE_THRESHOLD = 30;
      if (absDeltaY < MOUSE_THRESHOLD && absDeltaX < MOUSE_THRESHOLD) {
        return;
      }

      e.preventDefault();
      e.stopPropagation();

      const signY = Math.sign(e.deltaY);
      const signX = Math.sign(e.deltaX);
      const FIXED_TOTAL = e.shiftKey ? 100 : 150;
      const STEP_SIZE = 5;
      const MICRO_INTERVAL = 5;

      let remainY = FIXED_TOTAL * signY;
      let remainX = FIXED_TOTAL * signX;

      const meta = {
        clientX: e.clientX,
        clientY: e.clientY,
        screenX: e.screenX,
        screenY: e.screenY,
        ctrlKey: e.ctrlKey,
        altKey: e.altKey,
        metaKey: e.metaKey,
        shiftKey: false,
        deltaMode: e.deltaMode
      };

      function sendMicroStep() {
        let dY = 0, dX = 0;
        if (Math.abs(remainY) > 0) {
          const chunkY = Math.min(Math.abs(remainY), STEP_SIZE);
          dY = Math.sign(remainY) * chunkY;
          remainY -= Math.sign(remainY) * chunkY;
        }
        if (Math.abs(remainX) > 0) {
          const chunkX = Math.min(Math.abs(remainX), STEP_SIZE);
          dX = Math.sign(remainX) * chunkX;
          remainX -= Math.sign(remainX) * chunkX;
        }

        const newEvt = new WheelEvent("wheel", {
          deltaX: dX,
          deltaY: dY,
          deltaZ: 0,
          deltaMode: meta.deltaMode,
          clientX: meta.clientX,
          clientY: meta.clientY,
          screenX: meta.screenX,
          screenY: meta.screenY,
          ctrlKey: meta.ctrlKey,
          altKey: meta.altKey,
          metaKey: meta.metaKey,
          shiftKey: meta.shiftKey,
          bubbles: true,
          cancelable: true,
          composed: true,
          view: window
        });
        Object.defineProperty(newEvt, "_isCustomZoomEvent", {
          value: true,
          writable: false
        });
        e.target.dispatchEvent(newEvt);

        if (Math.abs(remainY) > 0 || Math.abs(remainX) > 0) {
          setTimeout(sendMicroStep, MICRO_INTERVAL);
        }
      }

      sendMicroStep();
    }, { capture: true, passive: false });
  }
  tryAttach();
})();

(function initLegendControls() {
  if (
    typeof datamap === "undefined" ||
    !datamap.metaData ||
    !document.getElementById("legend-toggle") ||
    !document.getElementById("legend-source") ||
    !document.getElementById("legend-topic") ||
    !document.getElementById("legend-ticker")
  ) {
    return setTimeout(initLegendControls, 100);
  }

  // 1) Prepare the data and elements
  const radios  = document.querySelectorAll('input[name="filter"]');
  const blocks  = {
    source:  document.getElementById("legend-source"),
    topic:   document.getElementById("legend-topic"),
    tickers: document.getElementById("legend-ticker")
  };
  // Sets of selected values for each category
  const selected = {
    source:  new Set(),
    topic:   new Set(),
    tickers: new Set()
  };

  // 2) The “AND”-filter function: returns an array of point indices,
  //    satisfying all non-zero sets selected.*
  function getCombinedIndices() {
    // If no filter is checked -> no filter at all
    if (
      selected.source.size === 0 &&
      selected.topic.size === 0 &&
      selected.tickers.size === 0
    ) {
      return [];
    }

    const md = datamap.metaData;
    const total = md.source.length;
    const result = [];

    for (let i = 0; i < total; i++) {
      // 2.1) Check by Source (if there is a choice)
      if (selected.source.size > 0 && !selected.source.has(md.source[i])) {
        continue;
      }
      // 2.2) Checking by Topic (if there is a choice)
      if (selected.topic.size > 0 && !selected.topic.has(md.topic[i])) {
        continue;
      }
      // 2.3) Check by Tickers (if there is a choice)
      if (selected.tickers.size > 0) {
        const raw = md.tickers[i];
        let list;
        if (Array.isArray(raw)) {
          list = raw;
        } else if (raw && typeof raw === "string") {
          list = raw.split(/\s*,\s*/);
        } else {
          list = [];
        }
        const ok = list.some(t => selected.tickers.has(t));
        if (!ok) continue;
      }
      // If all active checks have been passed ─ the point is suitable
      result.push(i);
    }
    return result;
  }

  // 3) Synchronize the visual appearance of “.box” boxes according to the selected.*
  function syncAllBoxes() {
    Object.keys(blocks).forEach(field => {
      blocks[field]
        .querySelectorAll(".box")
        .forEach(el => {
          const val = el.id;
          if (selected[field].has(val)) {
            el.textContent = "✓";
            el.style.opacity = "0.3";
          } else {
            el.textContent = "";
            el.style.opacity = "1.0";
          }
        });
    });
  }

  // 4) Render tags (only for the specified field) inside <div class=“selected-tags” data-field=“...”>
  function renderTagsForField(field) {
    const tagsContainer = document.querySelector(`.selected-tags[data-field="${field}"]`);
    if (!tagsContainer) return;
    tagsContainer.innerHTML = "";
    selected[field].forEach(val => {
      const tag = document.createElement("span");
      tag.classList.add("tag");
      tag.dataset.field = field;
      tag.dataset.value = val;

      const textSpan = document.createElement("span");
      textSpan.classList.add("tag-text");
      textSpan.textContent = val;

      const removeSpan = document.createElement("span");
      removeSpan.classList.add("tag-remove");
      removeSpan.textContent = "×";

      tag.appendChild(textSpan);
      tag.appendChild(removeSpan);
      tagsContainer.appendChild(tag);
    });
  }

  // 5) When switching tabs (radio change) just hide/show the desired block,
  //    synchronize squares and tags, and then do the merged filtering at once.
  function updateLegendField(field) {
    Object.entries(blocks).forEach(([k, el]) => {
      el.style.display = (k === field ? "block" : "none");
    });
    // Synchronize the squares in the active block
    syncAllBoxes();
    // Rendering tags in the active block
    renderTagsForField(field);

    // Updating the merged filtering
    const combined = getCombinedIndices();
    datamap.removeSelection("legend");
    if (combined.length) {
      datamap.addSelection(combined, "legend");
    }
    datamap.highlightPoints("legend");
  }

  // Putting a listener on the radio buttons
  radios.forEach(radio => {
    radio.addEventListener("change", () => {
      if (radio.checked) updateLegendField(radio.value);
    });
  });
  // Initialization of the first tab (“source”)
  const initField = document.querySelector('input[name="filter"]:checked').value;
  updateLegendField(initField);

  // 6) Search inside each block: hide/show .rows
  document.querySelectorAll(".filter-search").forEach(input => {
    input.addEventListener("input", () => {
      const field = input.dataset.field;
      const term = input.value.trim().toLowerCase();
      blocks[field].querySelectorAll(".row").forEach(row => {
        const label = row.querySelector(".item-label").textContent.toLowerCase();
        row.style.display = label.includes(term) ? "flex" : "none";
      });
    });
  });

  // 7) Click on the “.box” square: switch the value to selected[field], synchronize UI and filter
  document.addEventListener("click", event => {
    const box = event.target.closest(".box");
    if (!box) return;

    const field = document.querySelector('input[name="filter"]:checked').value;
    const block = blocks[field];
    if (!block.contains(box)) return;

    const val = box.id;
    const selSet = selected[field];
    if (selSet.has(val)) {
      selSet.delete(val);
    } else {
      selSet.add(val);
    }

    // Synchronize the appearance of squares and tags
    syncAllBoxes();
    renderTagsForField(field);

    // Recalculating the combined filtering
    const combined = getCombinedIndices();
    datamap.removeSelection("legend");
    if (combined.length) datamap.addSelection(combined, "legend");
    datamap.highlightPoints("legend");
  });

  // 8) Click on “×” inside a tag: uncheck, update tags, checkboxes and filtering
  document.addEventListener("click", event => {
    const rem = event.target.closest(".tag-remove");
    if (!rem) return;
    const tag = rem.parentElement;
    const field = tag.dataset.field;
    const val = tag.dataset.value;

    if (selected[field].has(val)) {
      selected[field].delete(val);
      // Uncheck the box if it is visible:
      const box = document.querySelector(
        `#legend-${field} .box[id="${CSS.escape(val)}"]`
      );
      if (box) {
        box.textContent = "";
        box.style.opacity = "1.0";
      }
    }

    // synchronize ALL squares in the current field
    syncAllBoxes();

    // Update tag display
    renderTagsForField(field);

    // Recalculate the “AND”-filtering and apply to the map
    const combined = getCombinedIndices();
    datamap.removeSelection("legend");
    if (combined.length) datamap.addSelection(combined, "legend");
    datamap.highlightPoints("legend");
  });
})();
"""

Declare a class that implements the functionality of lasso selection

In [11]:
import string
import datamapplot.selection_handlers
import sklearn.feature_extraction.text


class WordCloud(datamapplot.selection_handlers.SelectionHandlerBase):
    def __init__(self, n_words=256, width=300, height=300, stop_words=None, color_scale="YlGnBu", **kwargs):
        super().__init__(
            dependencies=[
                "https://d3js.org/d3.v6.min.js",
                "https://cdnjs.cloudflare.com/ajax/libs/d3-scale-chromatic/1.5.0/d3-scale-chromatic.min.js",
                "https://unpkg.com/d3-cloud@1.2.7/build/d3.layout.cloud.js",
            ],
            **kwargs
        )
        self.n_words = n_words
        self.width = width
        self.height = height
        self.stop_words = stop_words or list(sklearn.feature_extraction.text.ENGLISH_STOP_WORDS) + ['zacks', "simply", "wall", "st", "motley", "fool"]
        # Преобразуем имя палитры, например "YlGnBu" → "YlGnBu"
        self.color_scale = string.capwords(color_scale[:1]) + color_scale[1:]
        self.color_scale_reversed = False

    @property
    def javascript(self):
        return f"""
(function initWordCloudLasso() {{
  // Waiting for the DataMap and #word-cloud container to be ready:
  if (
    typeof datamap === "undefined" ||
    !datamap.metaData ||
    !document.getElementById("word-cloud")
  ) {{
    return setTimeout(initWordCloudLasso, 100);
  }}

  // === Word cloud parameters ===
  const _STOPWORDS = new Set({self.stop_words});
  const MIN_WORD_LEN = 3;
  const N_WORDS = {self.n_words};

  // break text into words, discard stop words and tokens that are too short.
  function processText(text) {{
    if (!text) return [];
    const cleaned = text.toLowerCase().replace(/[^\\w\\s]/g, ' ');
    const tokens = cleaned.match(/\\w+/g) || [];
    return tokens.filter(w => w.length >= MIN_WORD_LEN && !_STOPWORDS.has(w));
  }}

  // Count word frequencies and normalize sizes
  function wordCounter(textItems) {{
    const allWords = textItems.flatMap(processText);
    const counts = new Map();
    allWords.forEach(w => counts.set(w, (counts.get(w) || 0) + 1));

    let arr = Array.from(counts, ([text, freq]) => ({{ text: text, size: Math.sqrt(freq) }}));
    arr.sort((a, b) => b.size - a.size);
    arr = arr.slice(0, N_WORDS);
    const maxSize = Math.max(0, ...arr.map(x => x.size));
    return arr.map(({{ text, size }}) => ({{ text: text, size: size / maxSize }}));
  }}

  // Word cloud drawing function (D3-cloud)
  function generateWordCloud(words) {{
    // Remove the old SVG (if any)
    d3.select("#word-cloud").select("svg").remove();

    // These width/height values are only needed for the viewBox:
    const width = {self.width};
    const height = {self.height};

    // Create an svg container, set its viewBox so that the SVG can stretch to 100% of the #word-cloud container
    const svg = d3.select("#word-cloud")
      .append("svg")
        .attr("viewBox", `0 0 ${{width}} ${{height}}`)
        .attr("preserveAspectRatio", "xMidYMid meet")
      .append("g")
        .attr("transform", `translate(${{width / 2}},${{height / 2}})`);

    // Define the color scale inside the same function so that draw can “see” it
    const colorScale = d3.scaleSequential(d3.interpolate{self.color_scale})
      .domain([{ "0, width / 10" if not self.color_scale_reversed else "width / 10, 0" }]);

    // Configure layout: the dimensions of the virtual field (width×height) correspond to the viewBox
    const layout = d3.layout.cloud()
      .size([width, height])
      .words(words.map(d => ({{ text: d.text, size: 5 + d.size * width / 10 }})))
      .rotate(() => 0)
      .font("Montserrat")
      .fontSize(d => d.size)
      .on("end", draw);

    layout.start();

    // The function that will be called when the calculation of word positions is finished
    function draw(words) {{
      // Binding data to text elements
      const txt = svg.selectAll("text").data(words, d => d.text);
      txt.exit().remove();
      txt.enter()
        .append("text")
        .attr("text-anchor", "middle")
        .merge(txt)
        .text(d => d.text)
        .attr("transform", d => "translate(" + [d.x, d.y] + ")rotate(" + d.rotate + ")")
        .attr("font-size", d => d.size)
        .attr("fill", d => colorScale(d.size))
        .attr("font-family", "Montserrat");
    }}
  }}

  // ============ Lasso-callback ============
  let inLassoHandler = false;

  function lassoSelectionCallback(selectedPoints) {{
    if (inLassoHandler) return;
    inLassoHandler = true;

    const wcContainer = document.getElementById("word-cloud");

    // 1) Check to see if the lasso is removed without a real selection:
    const legendIndices = Array.isArray(window.legendFilteredIndices)
      ? window.legendFilteredIndices.slice().sort((a, b) => a - b)
      : [];
    const sp = Array.isArray(selectedPoints)
      ? selectedPoints.slice().sort((a, b) => a - b)
      : [];

    const isSameAsFiltered =
      sp.length === legendIndices.length &&
      sp.every((v, idx) => v === legendIndices[idx]);

    // 2) If “empty” or matches the legend filtering - REMOVE the cloud
    if (sp.length === 0 || isSameAsFiltered) {{
      // clean up the SVG content
      d3.select("#word-cloud").selectAll("*").remove();
      // deselect
      datamap.removeSelection("lasso-select");
      // remove the background by removing the .active class
      wcContainer.classList.remove("active");
      inLassoHandler = false;
      return;
    }}

    // 3) Otherwise - you got an incorrectly empty array? Let's check the words:
    const selectedText = sp.map(i => datamap.metaData.text[i]);
    const wordCounts = wordCounter(selectedText);

    // 4) If suddenly there are no words, we clean the cloud too
    if (wordCounts.length === 0) {{
      d3.select("#word-cloud").selectAll("*").remove();
      datamap.removeSelection("lasso-select");
      wcContainer.classList.remove("active");
      inLassoHandler = false;
      return;
    }}

    // 5) Now we draw a new cloud:
    generateWordCloud(wordCounts);

    // 6) After inserting <svg> with text - add the background
    wcContainer.classList.add("active");

    // 7) Update “lasso-select” (removed the old one, added a new one, highlighted it)
    datamap.removeSelection("lasso-select");
    datamap.addSelection(sp, "lasso-select");
    datamap.highlightPoints("lasso-select");

    inLassoHandler = false;
  }}

  // Registering our lasso-collback under the “lasso-select” key.
  (function registerLassoHandler() {{
    if (typeof datamap !== "undefined" && datamap.addSelectionHandler) {{
      datamap.addSelectionHandler(lassoSelectionCallback, "lasso-select");
    }} else {{
      setTimeout(registerLassoHandler, 100);
    }}
  }})();
}})();
"""

    @property
    def html(self):
        return """<div id="word-cloud" class="container-box"></div>"""

    @property
    def css(self):
        return f"""
/* === WORD CLOUD STYLES === */
#word-cloud {{
  position: absolute;
  left: 2vw;
  bottom: 20vh;
  width: 27.7778vw;
  aspect-ratio: 1 / 1;
  z-index: 10;
  background: transparent;
  transition: background 0.3s ease;
}}

#word-cloud.active {{
  background: rgba(0, 0, 0, 0.5);
}}

/* Internal SVG (D3-cloud) Spread across the entire parent's space. */
#word-cloud svg {{
  width: 100%;
  height: 100%;
}}
"""

# Vizualize Semantic Map

In [12]:
plot = datamapplot.create_interactive_plot(
    # WORING WITH DOTS
    df2[["x", "y"]].to_numpy(),
    df2["topic"],
    # For some reason, even though the design of popups is set elsewhere,
    # popups don't work without passing this argument
    hover_text=df2["title"],
    hover_text_html_template=hover_text_template,
    # Set the size of the points relative to the text length
    marker_size_array=df2["char_len"],
    point_radius_min_pixels=1.5,
    point_radius_max_pixels=5,
    # Remove the white outline around the dots
    point_line_width=0,
    # Making points clickable
    on_click="window.open(`{link}`)",
    # Customizing noise processing
    noise_label="Unlabelled",
    noise_color="#212121",

    # WORKING WITH CLUSTERS
    text_min_pixel_size=16,
    text_max_pixel_size=16,
    # To keep topic headings from colliding with each other
    text_collision_size_scale=4,
    # Set just white cluster names, not colored ones
    color_label_text=False,
    palette_hue_radius_dependence=10,
    cvd_safer=True,
    # In order to have the topic headings located in the cluster
    # median rather than in the middle of the cluster
    use_medoids=True,
    # Bypass the clusters with counterparts
    polygon_alpha=2,
    cluster_boundary_polygons=True,
    cluster_boundary_line_width=1,

    # GENERAL SETTINGS
    # title="Semantic Map of Financial News",
    # sub_title="Sample of 125,000 news articles from Yahoo! Finance",
    font_family="Montserrat",
    darkmode=True,
    initial_zoom_fraction=0.8,
    custom_css=custom_css,
    custom_html=custom_html,
    custom_js=custom_js,

    # ENABLE PREVIOSLY DEVELOPED FUNCTIONALITIES
    # We add auxiliary data for building popups, click redirect,
    # date breakdown in the bar graph, as well as text search
    # and word cloud building
    extra_point_data=df2[[
        "link", "title", "topic", "tickers",
        "source", "datetime_formatted", "text"]
    ],
    # Add hidden text search
    enable_search=True,
    search_field="text",
    # Add a word cloud
    selection_handler=WordCloud(n_words=256, color_scale="plasma"),
    # Add a histogram to filter by number of characters
    histogram_data=df2["datetime"],
    histogram_settings={
        "histogram_title":"Dates of Publication",
        "histogram_bin_fill_color":"#a64531",
        "histogram_bin_unselected_fill_color":"#d6a591",
        "histogram_bin_selected_fill_color":"#f68571",
        "histogram_width": 400,
        "histogram_height": 125,
    }
)

# Save Semantic Map
plot.save("semantic_map.html")